# Test alpha-spheres network 2

In [1]:
import molsysmt as msm

In [2]:
from scipy.spatial import Voronoi
from math import sqrt

In [3]:
from scipy.spatial.distance import euclidean

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [6]:
molecular_system = msm.convert('1HIV')
molecular_system = msm.extract(molecular_system, selection='molecule_type in ["protein", "peptide"]')

In [7]:
msm.info(molecular_system)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_proteins,n_structures
molsysmt.MolSys,1516,198,2,2,2,1,2,1


## Castp

In [21]:
pocket = []
with open('castp/1hiv/1hiv.poc') as file:
    for line in file:
        fields = line.rstrip().split()
        if fields[-2]=='1':
            pocket.append(int(fields[1]))

pocket = msm.get(molecular_system, element='atom', selection='atom_id==@pocket', atom_index=True)

In [36]:
view=msm.view(molecular_system)
view

NGLWidget()

In [37]:
view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

In [38]:
sel='@'+','.join([str(ii) for ii in pocket])
#view.add_surface(sel, color='red', opacity=0.3)
view.add_ball_and_stick(sel, color='red', opacity=0.3)

In [39]:
pocket

array([  66,   67,   68,   69,   70,   71,  180,  181,  196,  197,  201,
        206,  207,  208,  210,  211,  213,  214,  218,  219,  220,  221,
        222,  225,  226,  229,  243,  365,  367,  368,  369,  370,  371,
        372,  373,  374,  375,  376,  377,  378,  381,  382,  383,  401,
        402,  403,  585,  610,  616,  617,  618,  624,  625,  640,  641,
        661,  825,  826,  827,  828,  829,  830,  939,  940,  956,  960,
        965,  966,  967,  969,  970,  972,  973,  977,  978,  979,  980,
        981,  984,  985,  988, 1002, 1121, 1124, 1126, 1127, 1128, 1129,
       1130, 1131, 1132, 1133, 1136, 1140, 1141, 1143, 1161, 1162, 1370,
       1376, 1377, 1383, 1399, 1400, 1420])

# Mine

In [40]:
coordinates = msm.get(molecular_system, coordinates=True)

In [41]:
coordinates = msm.pyunitwizard.get_value(coordinates[0])
n_atoms = coordinates.shape[0]

In [42]:
radii = [0.2 for ii in range(n_atoms)]
probe = 0.1

In [43]:
voronoi = Voronoi(coordinates)

In [44]:
centers = voronoi.vertices
n_alphaspheres = centers.shape[0]

In [45]:
atoms_of_alphasphere = {ii:[] for ii in range(n_alphaspheres)}
atoms_of_alphasphere[-1] = []
alphaspheres_of_atom = {ii:[] for ii in range(n_atoms)}

In [46]:
for atom_index, region_index in enumerate(voronoi.point_region):
    for alphasphere_index in voronoi.regions[region_index]:
        atoms_of_alphasphere[alphasphere_index].append(atom_index)
        alphaspheres_of_atom[atom_index].append(alphasphere_index)

In [67]:
alphaspheres_in_castp = []
for ii in range(n_alphaspheres):
    if np.isin(atoms_of_alphasphere[ii], pocket).all():
        alphaspheres_in_castp.append(ii)

In [68]:
alphaspheres_in_castp[0]

385

In [69]:
atoms_of_alphasphere[385]

[616, 1129, 1130, 1161]

In [55]:
pocket

array([  66,   67,   68,   69,   70,   71,  180,  181,  196,  197,  201,
        206,  207,  208,  210,  211,  213,  214,  218,  219,  220,  221,
        222,  225,  226,  229,  243,  365,  367,  368,  369,  370,  371,
        372,  373,  374,  375,  376,  377,  378,  381,  382,  383,  401,
        402,  403,  585,  610,  616,  617,  618,  624,  625,  640,  641,
        661,  825,  826,  827,  828,  829,  830,  939,  940,  956,  960,
        965,  966,  967,  969,  970,  972,  973,  977,  978,  979,  980,
        981,  984,  985,  988, 1002, 1121, 1124, 1126, 1127, 1128, 1129,
       1130, 1131, 1132, 1133, 1136, 1140, 1141, 1143, 1161, 1162, 1370,
       1376, 1377, 1383, 1399, 1400, 1420])

In [116]:
edges=[]
interfaces={}

for ii in tqdm(range(n_alphaspheres)):
    
    neighbors=[]
    for jj in atoms_of_alphasphere[ii]:
        for kk in alphaspheres_of_atom[jj]:
            if kk>ii:
                neighbors.append(kk)
    neighbors=np.unique(neighbors)

    interfaces[ii]={}
    
    for jj in neighbors:
        common = np.intersect1d(atoms_of_alphasphere[ii], atoms_of_alphasphere[jj], assume_unique=True)
        if len(common)>=3:
            edges.append([ii,jj])
            interfaces[ii][jj]=common

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9810/9810 [00:03<00:00, 2539.12it/s]


In [117]:
len(edges)

19547

In [118]:
import networkx as nx

In [119]:
G = nx.Graph(edges)

In [120]:
G[385]

AtlasView({386: {}, 391: {}, 2972: {}, 8315: {}})

In [121]:
interfaces[385]

{386: array([ 616, 1129, 1161]),
 391: array([1129, 1130, 1161]),
 2972: array([ 616, 1130, 1161]),
 8315: array([ 616, 1129, 1130])}

In [156]:
face_coordinates = coordinates[interfaces[385][386]]

In [157]:
centroid = face_coordinates.mean(axis=1)

In [158]:
euclidean(centroid, face_coordinates[0])

1.8312079237486931

In [159]:
euclidean(centroid, face_coordinates[1])

1.725568204969018

In [160]:
euclidean(centroid, face_coordinates[2])

2.0939564345993444

In [153]:
view=msm.view(molecular_system)
view

NGLWidget()

In [154]:
view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

In [161]:
for ii in [385, 386]:
    aa=atoms_of_alphasphere[ii]
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[1]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[2]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)

In [ ]:
len(edges)

In [ ]:
def atoms_are_permeable(atom_indices, coordinates, radii, probe_radius):

    centroid = coordinates[atom_indices,:].mean(axis=0)

    permeable=True
    
    for atom_index in atom_indices:
        distance = euclidean(coordinates[atom_index], centroid)
        if distance < radii[atom_index]+probe_radius:
            permeable=False
            break

    return permeable

In [ ]:
edges=[]
permeability={}

for kk in tqdm(range(len(voronoi.ridge_vertices))):
    
    aux_ridge_vertices = np.sort(voronoi.ridge_vertices[kk])
    n = len(aux_ridge_vertices)

    for ii in range(n):
        aa = aux_ridge_vertices[ii]
        if aa!=-1:
            for jj in range(ii+1,n):
                bb = aux_ridge_vertices[jj]
                common = np.intersect1d(atoms_of_alphasphere[aa], atoms_of_alphasphere[bb], assume_unique=True)
                common = tuple(common)
                if len(common)>2:
                    if common not in permeability:
                        with_edge = atoms_are_permeable(common, coordinates, radii, probe)
                        permeability[common]=with_edge
                        if with_edge:
                            edges.append([aa,bb])

In [ ]:
len(edges)

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph(edges)

In [ ]:
G.number_of_nodes()

In [ ]:
n_alphaspheres

In [ ]:
pockets = list(nx.connected_components(G))

In [ ]:
len(pockets)

In [ ]:
view=msm.view(molecular_system)
view

In [ ]:
view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

In [ ]:
for ii in range(len(pockets)):
    if len(pockets[ii])>10:
        print(ii, len(pockets[ii]))

In [ ]:
pocket = 40
alphaspheres_in_pocket = pockets[pocket]
atoms_in_pocket = np.concatenate([atoms_of_alphasphere[ii] for ii in alphaspheres_in_pocket])
atoms_in_pocket = np.unique(atoms_in_pocket)

print(alphaspheres_in_pocket)

In [ ]:
sel='@'+','.join([str(ii) for ii in atoms_in_pocket])
view.add_surface(sel, color='red', opacity=0.3)

In [ ]:
for ii in alphaspheres_in_pocket:
    aa=atoms_of_alphasphere[ii]
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[1]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[2]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)